In [2]:
import pygad 
import pandas as pd
import numpy as np

In [113]:
data = np.loadtxt('sim_prices.csv',delimiter=',')

In [117]:
prices = data.reshape(1001,120,2)
bond_prices1 = prices[0,:,1] 

In [119]:
bond_returns = bond_prices1[:-1]/bond_prices1[1:]-1


In [106]:
means = np.sort(returns[:,:,1].mean(1)) * 12

In [97]:
bonds = returns[:,:,0]
means = np.sort((bonds+1).cumprod(1)[:,-1])

In [15]:
class Simulation():
    def __init__(self, initial_value, inflows, returns, strategy) -> None:
        self.capital = initial_value
        self.returns = returns
        self.strategy = strategy 
        self.inflows = inflows
    
   
    def step(self, current_value,inflow,weights,returns):
        current_assets_value = np.expand_dims(current_value,axis=0).T * weights
        next_assets_value =  np.round(current_assets_value * (1+returns),2)
        return next_assets_value.sum(1)
    
        
    def run(self, T):
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],self.strategy[t],self.returns[:,t])


In [30]:
portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,1.0]]

In [101]:
portfolio_sim = Simulation(100,np.full(10,1),returns, np.repeat(portfolios[2],120))


In [105]:
portfolio_sim.run(10)
np.quantile(portfolio_sim.capital,0.9)


0.0